# Noether Checks

Check that the BdNMC runs on Noether performed nominally

In [3]:
import uproot3 as up3
import numpy as np
import pandas as pd
import os

* Checks for missing .root files
* Checks for missing .log / .out / .err / .sub files
    * Check .sub file matches .root file params
* Reads all the .log / .out / .err files for each job, checks for errors
* Reads the root files, checks
    * N_chi > 10_000
    * Ma / mchi = dt ratio of file
    * Anything else


### Check For Missing .ROOT Files

In [4]:
# get a list of the card files that I used to generate the data
card_list_files = []
for file in os.listdir("../Noether"):
    if "pi0" in file:
        card_list_files.append(file)
    elif "eta" in file:
        card_list_files.append(file)
    else:
        continue

print(card_list_files)

# read these files and look for a root file in data/root/bdnmc that has the same name
for file in card_list_files:
    with open(f"../Noether/{file}") as f:
        lines = f.readlines()
        for line in lines:
            root_file_name = line[:-5] + ".root"
            if not(os.listdir(f"../data/root/BdNMC/{root_file_name}")):
                print(root_file_name, " not found!")
            
            # could have a look inside the root file here too
            with up3.open(f"../data/root/BdNMC/{root_file_name}") as f:
                trees = f.keys()
                

['card_list_fermion_pi0_0.6.txt', 'card_list_scalar_eta_0.6.txt', 'card_list_fermion_eta_0.33.txt', 'card_list_fermion_eta_0.6.txt', 'card_list_scalar_pi0_0.33.txt', 'card_list_scalar_pi0_2.0.txt', 'card_list_scalar_eta_0.33.txt', 'card_list_scalar_eta_2.0.txt', 'card_list_fermion_pi0_2.0.txt', 'card_list_scalar_pi0_0.6.txt', 'card_list_fermion_pi0_0.33.txt', 'card_list_fermion_eta_2.0.txt']


FileNotFoundError: [Errno 2] No such file or directory: '../data/root/BdNMC/fermion/pi0_fermion_ma_0.01_dt_0.6.root'

### Check the log/out/err/sub files

In [1]:
PATH = "../data/Noether_25_03/"

In [5]:
f_sub = []
f_out = []
f_err = []
f_log = []
# loop through all files in the directory
for file in os.listdir(PATH):
    if file.endswith(".sub"):
        f_sub.append(file)
    elif file.endswith(".out"):
        f_out.append(file)
    elif file.endswith(".err"):
        f_err.append(file)
    elif file.endswith(".log"):
        f_log.append(file)
    else:
        print("Unknown file type: ", file)

Unknown file type:  scalar
Unknown file type:  fermion


In [24]:
mismatch = {"fermion_pi0" : pd.DataFrame(columns=["0.33", "0.6", "2.0"]), 
            "fermion_eta" : pd.DataFrame(columns=["0.33", "0.6", "2.0"]),
            "scalar_pi0" : pd.DataFrame(columns=["0.33", "0.6", "2.0"]),
            "scalar_eta" : pd.DataFrame(columns=["0.33", "0.6", "2.0"])}

for subf in f_sub:
    try:
        _, meson, chi_type, _, ma, _, dt = subf.split("_")
        dt = dt[:-4] # remove the .sub
    except ValueError:
        continue
    with open(PATH + subf) as f:
        for line in f:
            if "arguments" in line:
                meson_arg, chi_type_arg, _, ma_arg, _, dt_arg = line.split("/")[1].split("_")
                dt_arg = dt_arg[:-5]
            if "output" in line:
                _, meson_out, chi_type_out, _, ma_out, _, dt_out = line.split("/")[1].split("_")
    
    if meson != meson_arg or chi_type != chi_type_arg or ma != ma_arg or dt != dt_arg:
        #print("Mismatch in arguments: ", meson, meson_arg, chi_type, chi_type_arg, ma, ma_arg, dt, dt_arg)
        mismatch[chi_type + "_" + meson].loc[float(ma), dt] = "A"
    if meson != meson_out or chi_type != chi_type_out or ma != ma_out or dt != dt_out:
        mismatch[chi_type + "_" + meson].loc[float(ma), dt] = "O"
    else:
        mismatch[chi_type + "_" + meson].loc[float(ma), dt] = "."

for i in mismatch:
    print(i)
    #display(mismatch[i].replace(np.nan, "_").sort_index(ascending=True))
    #print("key:\n> A = argument mismatch\n> O = output mismatch\n> . = match\n _ = missing this file")


fermion_pi0
fermion_eta
scalar_pi0
scalar_eta


### Noether_25_03 ROOT checks

In [47]:
def check_root_file_name_ma_dt(path):
    incorrect_names = []
    chi_types = ["fermion", "scalar"]
    for c_type in chi_types:
        for file in os.listdir(path+c_type+"/"):
            if file.endswith(".root"):
                #print(file)
                with up3.open(path+c_type+"/"+file) as f:
                    trees = f.keys()
                    if trees == []:
                        print("No trees in file "+file)
                        continue
                    dm_mass = f[b'model_tree'][b'dm_mass'].array()
                    dp_mass = f[b'model_tree'][b'dp_mass'].array()
                    dt = dm_mass / dp_mass

                    dt_of_file = float(file.split("_")[-1][:-5])
                    ma_of_file = float(file.split("_")[-3])
                    if dt_of_file != round(dt[0],2):
                        print("Mismatch in dt: ", dt_of_file, round(dt[0],2))
                        incorrect_names.append(file)
                    
                    if ma_of_file != round(dp_mass[0],2):
                        print("Mismatch in ma: ", ma_of_file, round(dp_mass[0],2))
                        incorrect_names.append(file)
    
    return incorrect_names

fs = check_root_file_name_ma_dt("../data/Noether_25_03/")
if len(fs) == 0:
    print("\n!! All root files are named correctly !!")
else:
    print("\n!! Some root files are named incorrectly !!")
    print(fs)

No trees in file eta_fermion_ma_0.06_dt_0.33.root
No trees in file eta_fermion_ma_0.03_dt_0.33.root
No trees in file eta_fermion_ma_0.05_dt_0.33.root
No trees in file eta_fermion_ma_0.01_dt_0.33.root
No trees in file pi0_fermion_ma_0.01_dt_0.33.root
No trees in file eta_fermion_ma_0.07_dt_0.33.root
No trees in file pi0_fermion_ma_0.04_dt_0.33.root
No trees in file eta_fermion_ma_0.09_dt_0.33.root
No trees in file pi0_fermion_ma_0.02_dt_0.33.root
No trees in file eta_fermion_ma_0.08_dt_0.33.root
No trees in file pi0_fermion_ma_0.03_dt_0.33.root
No trees in file eta_fermion_ma_0.01_dt_0.6.root
No trees in file eta_fermion_ma_0.1_dt_0.33.root
No trees in file eta_fermion_ma_0.04_dt_0.33.root
No trees in file eta_fermion_ma_0.02_dt_0.33.root
No trees in file pi0_scalar_ma_0.07_dt_2.0.root
No trees in file pi0_scalar_ma_0.08_dt_2.0.root
No trees in file pi0_scalar_ma_0.04_dt_2.0.root
No trees in file pi0_scalar_ma_0.09_dt_2.0.root
No trees in file pi0_scalar_ma_0.05_dt_2.0.root
No trees in 